In [ ]:
import pandas as pd
df = pd.read_csv('datasets/embedded_dataset.csv')
pd.set_option('display.max_rows', None)
df.head()

,customer_id,timestamp,action
0,1319,5/19/2022 18:16,transaction_imps
1,830,1/23/2022 20:11,priority_banking_request
2,1483,12/3/2020 9:07,FOR_add_beneficiary
3,1496,12/27/2022 20:25,transaction_imps
4,1213,9/10/2021 22:05,gold_loan_renew


In [4]:
# Sequential LSTM with checkpoint

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

from keras.callbacks import ModelCheckpoint

# Define the path where you want to save the model checkpoints
checkpoint_path = '/checkpoint_{epoch:02d}.h5'

# Create a ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,          # Path to save the model
    save_best_only=False,              # Set to True if you only want to save the best model
    save_weights_only=True,            # Set to True if you only want to save model weights
    period=5                           # Save every 5 epochs (adjust this interval as needed)
)

# One-hot encode the 'action' labels
df_encoded = pd.get_dummies(df, columns=['action'], prefix='action')

# Create sequences for each customer
sequence_length = 5  # Define the sequence length
sequences = []

for customer_id, group in df_encoded.groupby('customer_id'):
    actions = group.drop(['customer_id', 'timestamp'], axis=1).values
    for i in range(len(actions) - sequence_length + 1):
        sequence = actions[i:i+sequence_length]
        sequences.append(sequence)

# Split sequences into input and target
X = [sequence[:-1] for sequence in sequences]
y = [sequence[-1] for sequence in sequences]

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a sequential model
model = Sequential()

# Add an LSTM layer
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))

# Add a Dense output layer
model.add(Dense(34, activation='softmax'))  # Use 34 for multiple action prediction

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[model_checkpoint])

Epoch 1/50
21805/21805 [==============================] - 139s 6ms/step - loss: 3.5272 - accuracy: 0.0290 - val_loss: 3.5268 - val_accuracy: 0.0291
Epoch 2/50
21805/21805 [==============================] - 202s 9ms/step - loss: 3.5267 - accuracy: 0.0295 - val_loss: 3.5267 - val_accuracy: 0.0291
Epoch 3/50
21805/21805 [==============================] - 213s 10ms/step - loss: 3.5265 - accuracy: 0.0305 - val_loss: 3.5270 - val_accuracy: 0.0290
Epoch 4/50
21805/21805 [==============================] - 135s 6ms/step - loss: 3.5262 - accuracy: 0.0309 - val_loss: 3.5272 - val_accuracy: 0.0292
Epoch 5/50
21799/21805 [============================>.] - ETA: 0s - loss: 3.5258 - accuracy: 0.0316

PermissionError: [Errno 13] Unable to create file (unable to open file: name = '/checkpoint_05.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 302)

In [ ]:
# Resume checkpoint training

from keras.models import load_model

# Load the model
model = load_model('checkpoint_50.h5')

# Continue training
model.fit(X_train, y_train, epochs=50)

In [4]:
# LSTM Sequential with Pytorch

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# One-hot encode the 'action' labels
df_encoded = pd.get_dummies(df, columns=['action'], prefix='action')

# Sort the dataset by 'customer_id' and 'timestamp'
df_encoded = df_encoded.sort_values(by=['customer_id', 'timestamp'])

# Create sequences for each customer
sequence_length = 5  # Define the sequence length
sequences = []

for customer_id, group in df_encoded.groupby('customer_id'):
    actions = group.drop(['customer_id', 'timestamp'], axis=1).values
    for i in range(len(actions) - sequence_length + 1):
        sequence = actions[i:i+sequence_length]
        sequences.append(sequence)

# Split sequences into input and target
X = [sequence[:-1] for sequence in sequences]
y = [sequence[-1] for sequence in sequences]

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for training and testing
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 32  # You can adjust this as needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define a simple LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model and move it to the GPU
input_size = X_train.shape[2]
hidden_size = 128
num_classes = y_train.shape[1]  # Adjust based on the number of classes in 'action'

model = LSTMClassifier(input_size, hidden_size, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()

    # Evaluation on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}, Test Accuracy: {accuracy:.2f}%')

# Save the model
torch.save(model.state_dict(), '2M_model.pth')

Epoch 1, Test Accuracy: 70.91%
Epoch 2, Test Accuracy: 71.07%
Epoch 3, Test Accuracy: 70.95%
Epoch 4, Test Accuracy: 71.15%
Epoch 5, Test Accuracy: 70.98%
Epoch 6, Test Accuracy: 71.12%
Epoch 7, Test Accuracy: 71.12%
Epoch 8, Test Accuracy: 70.99%
Epoch 9, Test Accuracy: 71.15%
Epoch 10, Test Accuracy: 71.16%
